## Adsorbing and manipulating molecules onto graphene

In [1]:
from nanofilm import AtomType,Atom,Molecule,Slab,Film   # Imports NanoFilm's modules

#### Sets the AtomType objects (specifiying the pseudpotential files for later use in ab initio calculations)

In [2]:
hydrogen=AtomType(symbol="H",element="hydrogen",atomicnumber=1,atomicmass=1.007,pseudopotential="H.pbe.UPF")
carbon=AtomType("C","carbon",6,12.0107,"C.pbe.UPF")

#### Creates first H2 molecule from scratch

In [3]:
mol1=Molecule(moltype="H2")                    # H2 molecule, initially empty (no atoms)
h1=Atom(atomtype=hydrogen,x=[-0.44,0.0,0.0])   # First H atom
h2=h1.duplicate_atom()                         # Second H atom
h2.x=[0.44,0.0,0.0]                            # Changes the coordinates of the second H atom
mol1.add_atom(h1)                              # Adds the first H atom to the molecule
mol1.add_atom(h2)                              # Adds the second H atom to the molecule

#### Creates the substrate (Slab object) taking the coordinates from an .xyz file

In [4]:
slab=Slab(slabname="graphene",filename="graphene.xyz")   # Slab object (in this case, graphene unit cell, with coordinates
                                                         # taken from an XYZ file)
slab.add_ads_site(ads_site="top",pos=[1.23,0.0])         # Defines the first adsorption site ("top")
slab.add_ads_site("hollow",[1.23,1.41])                  # Defines the second adsorption site ("hollow")            
slab.add_ads_site("bridge",[0,1.42])                     # Defines the third adsorption site ("bridge")
slab.replicate_slab(4,3,True)                            # Replicates the slab in the XY plane, including the adsorption sites

#### Adsorbs the H2 molecule over a "hollow" site on the substrate (creating first a Film object)

In [5]:
film=Film()                                        # Initially empty Film object
film.mindistance=0.75                              # Minimum separation between the molecule and the substrate
film.set_substrate(slab)                           # Sets graphene as the substrate
film.add_molecule(mol1,"hollow",8,vert_sep=0.75)   # Places the H2 molecule over the eighth "hollow" site on the substrate
film.write_xyz("film1.xyz")                        # Writes an XYZ file
film.write_pw_input(filename='film1.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])            # Creates a basic input file for pw.x from Quantum Espresso package

#### Rotates the H2 molecule and enforces a minimum distance from the substrate to prevent overlap

In [6]:
mol1.rotate_molecule(theta=90,phi=45,psi=0)   # Rotates the H2 molecule according to the Euler angles theta, phi, and psi
film.enforce_mindist()                        # Required to enforce that the molecule will not be too close to the substrate
film.write_xyz("film2.xyz")
film.write_pw_input(filename='film2.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])

#### Duplicates the H2 molecule, placing it over the seventh "top" site on the substrate

In [7]:
mol2=mol1.duplicate_molecule()
film.add_molecule(mol2,"top",7,vert_sep=1.0)   # Places the second H2 molecule over the first "top" site on the substrate
mol2.rotate_molecule(theta=15,phi=30,psi=45)   # Rotates the second H2 molecule
film.enforce_mindist()
film.write_xyz("film3.xyz")
film.write_pw_input(filename='film3.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])

#### Removes both H2 molecules and adsorbs a benzene molecule at an arbitrary position on the substrate

In [8]:
film.remove_molecule(0)                               # Removes the first H2 molecule, with id=0
film.remove_molecule(1)                               # Removes the second H2 molecule, with id=1
mol3=Molecule("Benzene","benzene.xyz")                # Benzene molecule (coordinates taken from an XYZ file)
film.mindistance=2.0                                  # Redefines the minimum molecule-substrate separation
film.add_molecule(mol3,pos=[2.12,4.07],vert_sep=2.0)  # Benzene molecule is placed at an arbitrary location
film.write_xyz("film4.xyz")
film.write_pw_input(filename='film4.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])

#### Rotates the benzene molecule and demonstrates how to prevent molecule-substrate spurious overlap

In [9]:
mol3.rotate_molecule(30,30,30)
film.write_xyz("film5.xyz")
film.write_pw_input(filename='film5.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])          # Since the molecule overlaps with the substrate, you will likely to 
                                                 # have problems if you try to run pw.x with this input file
film.enforce_mindist()
film.write_xyz("film6.xyz")
film.write_pw_input(filename='film6.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])          # This input file, in turn, should work fine, since a minimum distance 
                                                 # separating the molecule and the substrate was enforced

#### Makes a copy of the benzene molecule, places it over the seventh "bridge" site and then displaces it

In [10]:
mol4=mol3.duplicate_molecule()
film.add_molecule(mol4,"bridge",7,vert_sep=2.0)
film.write_xyz("film7.xyz")
film.write_pw_input(filename='film7.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])
mol4.displace_molecule([1.0,1.5,-5.0])
film.write_xyz("film8.xyz")
film.write_pw_input(filename='film8.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])          # Since the molecule overlaps with the substrate, you will likely to 
                                                 # have problems if you try to run pw.x with this input file
film.enforce_mindist()
film.write_xyz("film9.xyz")                      
film.write_pw_input(filename='film9.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])          # This input file, in turn, should work fine, since a minimum distance 
                                                 # separating the molecule and the substrate was enforced

#### Replicates the slab and adds a TCNQ molecule, placing it over the "top" site 16

In [11]:
slab.replicate_slab(2,1,True)                 # Replicates the slab along X direction, including the adsorption sites
nitrogen=AtomType("N","nitrogen",16,14.0067)  # Create a new AtomType object, for nitrogen, since TCNQ has nitrogen in 
                                              # its composition
mol5=Molecule("TCNQ","tcnq.xyz")              # TCNQ molecule (coordinates taken from an XYZ file)
film.add_molecule(mol5,"top",16,vert_sep=3)
mol5.rotate_molecule(psi=90)                  # Rotates the molecule by 90 degrees around the Z axis
film.write_xyz("film10.xyz")
film.write_pw_input(filename='film10.in',
                    ecutwfc=22.0,ecutrho=88.0,
                    nspin=2,input_dft='vdw-df',
                    kvec=[4,4,4,0,0,0])